En este notebook voy a implementar un modelo de regresión logística aummentando los predictores, además de los rendimientos rezagados, con los indicadores RSI, SMA, MACD y BB.  
También se agregan algunas estrategias que se irán explicando en los comentarios

In [3]:
# Se realizan las importaciones necesarias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
# Función para calcular indicadores técnicos
def calculate_technical_indicators(df, price_col="price"):
    # SMA (Simple Moving Average) - Media móvil simple (ventana de 20 periodos)
    df["SMA"] = df[price_col].rolling(window=20).mean()

    # MACD (Moving Average Convergence Divergence)
    exp1 = df[price_col].ewm(span=12, adjust=False).mean()  # EMA rápida
    exp2 = df[price_col].ewm(span=26, adjust=False).mean()  # EMA lenta
    df["MACD"] = exp1 - exp2
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()  # Línea de señal

    # RSI (Relative Strength Index)
    delta = df[price_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # Bandas de Bollinger (ventana de 20 periodos, 2 desviaciones estándar)
    df["BB_Middle"] = df[price_col].rolling(window=20).mean()
    df["BB_Std"] = df[price_col].rolling(window=20).std()
    df["BB_Upper"] = df["BB_Middle"] + 2 * df["BB_Std"],
    df["BB_Lower"] = df["BB_Middle"] - 2 * df["BB_Std"]

    return df


In [9]:
# función para cargar los datos
def carga_archivo(archivo, tipo):
    data = pd.read_csv(archivo)
    if tipo == "b":
        data = data.rename(columns = {"Close time": "time", "Close": "price"})
    data["time"] = pd.to_datetime(data["time"])
    data.set_index("time", inplace=True)
    return data


In [11]:
# seleccionar el archivo de datos
archivo = "five_minute.csv"
tipo = "t"

In [12]:
"""Cargar el archivo de datos
Voy a realizar cálculos con diferentes archivos de datos que están en el directorio actual y corresponden a
datos del tutorial y a datos de Binance.
Los datos del tutorial están en el archivo "five_minute.csv" y los datos de Binance están en  "EURUSDT_2020_5MIN.csv" """

df = carga_archivo(archivo, tipo)


In [13]:
df

,price
time,
2019-01-01 22:00:00+00:00,1.146580
2019-01-01 22:05:00+00:00,1.146350
2019-01-01 22:10:00+00:00,1.146320
2019-01-01 22:15:00+00:00,1.146320
2019-01-01 22:20:00+00:00,1.146530
...,...
2019-12-30 23:35:00+00:00,1.120180
2019-12-30 23:40:00+00:00,1.120210
2019-12-30 23:45:00+00:00,1.120295
